In [ ]:
import asyncio
import logging
import sys
import model
import os

from aiogram import Bot, Dispatcher
from aiogram.client.default import DefaultBotProperties
from aiogram.enums import ParseMode
from aiogram.filters import CommandStart
from aiogram.types import Message, FSInputFile
from aiogram import F

TOKEN = '7455470086:AAE4QbNm5Rh7IjvyK6LEOFQEd-TA97DNVt0'    # токен бота
lastMessage = ""

dp = Dispatcher()

model_path = model.model_path
output_images_path = model.output_images_path

# Хендлер на команду /start
@dp.message(CommandStart())
async def command_start_handler(message: Message) -> None:
    user_data = message.from_user.full_name
    greeting_message = (f"Здравствуйте, {user_data}!")
    await message.answer(f'{greeting_message}')
    await message.answer( "Загрузите фотографию сварного шва, чтобы обнаружить дефекты."
                          " Фотографии будут обрабатываться по очереди."
                          " На обработку может уйти некоторое время.")
    await message.bot.unpin_all_chat_messages(message.chat.id)
    await message.bot.pin_chat_message(message.chat.id, message.message_id + 2, True)


# Хендлер на прием фотографии
@dp.message(F.photo)
async def picture_handler(message: Message, bot: Bot) -> None:
    current_picture_name = message.photo[-1].file_unique_id + '.jpg'
    global lastMessage
    await message.bot.download(file=message.photo[-1].file_id, destination = 'data/' + current_picture_name)

    waitMessage = "Идет обработка, ждите..."
    if lastMessage != waitMessage:
        lastMessage = waitMessage
        await bot.send_message(message.chat.id, waitMessage)

    input_image_path = 'data/' + current_picture_name

    answer_path = model.picture_handling(input_image_path, model_path, output_images_path)
    picture = FSInputFile(path=answer_path)
    await bot.send_photo(message.chat.id, photo=picture)
    os.remove('data/' + current_picture_name)
    lastMessage = "pic"


#Хендлер на прием фотографии файлом
@dp.message(F.document)
async def picture_file_handler(message: Message, bot: Bot) -> None:
    current_picture_name = message.document.file_unique_id + '.jpg'
    global lastMessage
    await message.bot.download(file=message.document.file_id, destination = 'data/' + current_picture_name)

    waitMessage = "Идет обработка, ждите..."
    if lastMessage != waitMessage:
        lastMessage = waitMessage
        await bot.send_message(message.chat.id, waitMessage)

    input_image_path = 'data/' + current_picture_name


    answer_path = model.picture_handling(input_image_path, model_path, output_images_path)
    picture = FSInputFile(path=answer_path)
    await bot.send_photo(message.chat.id, photo=picture)
    os.remove('data/' + current_picture_name)
    lastMessage = "pic"

# Хендлер на прием видео
@dp.message(F.video)
async def picture_handler(message: Message, bot: Bot) -> None:
    await bot.send_message(message.chat.id, "Извините, обработка видео не поддерживается")


# Хендлер на прием анимации
@dp.message(F.animation)
async def picture_handler(message: Message, bot: Bot) -> None:
    await bot.send_message(message.chat.id, "Извините, обработка анимаций не поддерживается")


async def main() -> None:
    bot = Bot(token=TOKEN, default=DefaultBotProperties(parse_mode=ParseMode.HTML))
    await dp.start_polling(bot)


if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, stream=sys.stdout)
    asyncio.run(main())

In [ ]:
import torch
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import ultralytics
import os
import pathlib

temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

model_path = 'AI_model/best_92.pt'
output_labels_path = 'outputs/labels'
output_images_path = 'outputs/images'

model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)

def predict(image_path, model_path, conf_threshold=0.35, iou_threshold=0.45):
  
    model.conf = conf_threshold  # порог уверенности
    model.iou = iou_threshold    # порог NMS
    
    img = Image.open(image_path)
    results = model(img)
    # results.show()
    preds = results.xyxy[0].cpu().numpy()

    label_path = os.path.join(output_labels_path, os.path.splitext(os.path.basename(image_path))[0] + '.txt')
    with open(label_path, 'w') as f:
        for pred in preds:
            x1, y1, x2, y2, conf, cls = pred
            # Конвертация координат в формат YOLO
            img_width, img_height = img.size
            x_center = (x1 + x2) / 2 / img_width
            y_center = (y1 + y2) / 2 / img_height
            width = (x2 - x1) / img_width
            height = (y2 - y1) / img_height
            f.write(f'{int(cls)} {x_center} {y_center} {width} {height}\n')
    print(f'Saved YOLO labels to {label_path}')
    return results

def picture_handling(input_image_path, model_path, output_images_path):
    results = predict(input_image_path, model_path)

    annotated_img = results.render()[0]
    annotated_img = Image.fromarray(annotated_img)
    annotated_img_path = os.path.join(output_images_path, os.path.basename(input_image_path))
    annotated_img.save(annotated_img_path)
    print(f'Saved annotated image to {annotated_img_path}')
    
    return (annotated_img_path)